In [2]:
import requests
from collections import defaultdict
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
from tqdm import tqdm

load_dotenv()

# ----------------- 參數設定 -----------------
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
WBTC_CONTRACT = "0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599"
TOP_N = 20  # 前 N 名波段型投資人
MIN_HOLD_DAYS = 1
MAX_HOLD_DAYS = 90
MAX_TRADES_THRESHOLD = 200  # 過多交易視為 bot 或 CEX

# ----------------- 取得區塊號 by timestamp -----------------
def get_block_by_timestamp(timestamp):
    url = "https://api.etherscan.io/api"
    params = {
        "module": "block",
        "action": "getblocknobytime",
        "timestamp": str(timestamp),
        "closest": "before",
        "apikey": ETHERSCAN_API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data["status"] != "1":
        raise Exception(f"無法取得區塊號: {data.get('message')}")
    return int(data["result"])

# ----------------- 抓取 ERC-20 交易 -----------------
def get_erc20_transactions(contract_address, start_block, end_block):
    url = "https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": contract_address,
        "startblock": start_block,
        "endblock": end_block,
        "sort": "asc",  # 時間順序，方便配對買賣
        "apikey": ETHERSCAN_API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data["status"] != "1":
        print(f"API 錯誤或沒有資料 ({contract_address}):", data.get("message"))
        return []
    return data["result"]

# ----------------- 分段抓取一年資料 -----------------
def fetch_last_year_transactions(contract_address):
    all_txs = []
    now = datetime.utcnow()
    one_year_ago = now - timedelta(days=365)

    # 每個月分段
    current_start = one_year_ago
    while current_start < now:
        current_end = current_start + timedelta(days=30)
        if current_end > now:
            current_end = now

        print(f"抓取 {current_start.date()} ~ {current_end.date()} 交易...")
        start_block = get_block_by_timestamp(int(current_start.timestamp()))
        end_block = get_block_by_timestamp(int(current_end.timestamp()))

        txs = get_erc20_transactions(contract_address, start_block, end_block)
        all_txs.extend(txs)

        current_start = current_end  # 下一段

    print(f"總共抓取 {len(all_txs)} 筆交易資料")
    return all_txs

# ----------------- 尋找波段型投資人 -----------------
def find_swing_traders(transactions, min_days=1, max_days=90, max_trades=200):
    addr_trades = defaultdict(list)

    # 將所有交易按地址分組
    for tx in tqdm(transactions):
        ts = datetime.utcfromtimestamp(int(tx["timeStamp"]))
        from_addr = tx["from"].lower()
        to_addr = tx["to"].lower()
        value = int(tx["value"]) / (10 ** int(tx["tokenDecimal"]))  # WBTC 單位
        if value <= 0:  # 跳過 0 交易
            continue
        addr_trades[from_addr].append(("sell", ts, value))
        addr_trades[to_addr].append(("buy", ts, value))

    swing_investors = []

    for addr, trades in addr_trades.items():
        if len(trades) > max_trades:
            continue  # 過濾掉高頻交易者

        trades.sort(key=lambda x: x[1])  # 按時間排序
        hold_times = []
        last_buy = None

        for action, ts, value in trades:
            if action == "buy":
                last_buy = ts
            elif action == "sell" and last_buy:
                days = (ts - last_buy).days
                if min_days <= days <= max_days:
                    hold_times.append(days)
                last_buy = None  # 重置

        if hold_times:
            avg_hold = sum(hold_times) / len(hold_times)
            swing_investors.append((addr, avg_hold, len(hold_times)))

    swing_investors.sort(key=lambda x: x[1], reverse=True)
    return swing_investors

# ----------------- 主程式 -----------------
if __name__ == "__main__":
    print("分段抓取 WBTC 過去一年交易中...")
    wbtc_txs = fetch_last_year_transactions(WBTC_CONTRACT)

    if wbtc_txs:
        swing_traders = find_swing_traders(wbtc_txs, MIN_HOLD_DAYS, MAX_HOLD_DAYS, MAX_TRADES_THRESHOLD)
        print(f"前 {TOP_N} 名波段型 WBTC 投資人 (最近一年):")
        for addr, avg_days, trades_count in swing_traders[:TOP_N]:
            print(f"{addr} -> 平均持有 {avg_days:.1f} 天，共 {trades_count} 筆波段交易")
    else:
        print("未抓取到 WBTC 交易資料")


分段抓取 WBTC 過去一年交易中...
抓取 2024-08-14 ~ 2024-09-13 交易...
抓取 2024-09-13 ~ 2024-10-13 交易...
抓取 2024-10-13 ~ 2024-11-12 交易...
抓取 2024-11-12 ~ 2024-12-12 交易...
抓取 2024-12-12 ~ 2025-01-11 交易...
抓取 2025-01-11 ~ 2025-02-10 交易...
抓取 2025-02-10 ~ 2025-03-12 交易...
抓取 2025-03-12 ~ 2025-04-11 交易...
抓取 2025-04-11 ~ 2025-05-11 交易...
抓取 2025-05-11 ~ 2025-06-10 交易...
抓取 2025-06-10 ~ 2025-07-10 交易...
抓取 2025-07-10 ~ 2025-08-09 交易...
抓取 2025-08-09 ~ 2025-08-14 交易...
總共抓取 130000 筆交易資料


100%|██████████| 130000/130000 [00:00<00:00, 332693.60it/s]

前 20 名波段型 WBTC 投資人 (最近一年):
0x86b792e6a20c8e8ef56ff4fc92aedcb62dbeefed -> 平均持有 90.0 天，共 1 筆波段交易
0xf93191d350117723dbeda5484a3b0996d285cecf -> 平均持有 90.0 天，共 1 筆波段交易
0xa717c7cec5fbab72aa9427777b8b40b2ce27f4b6 -> 平均持有 90.0 天，共 1 筆波段交易
0x57d0175622948dc647768d17d818b93331668d91 -> 平均持有 90.0 天，共 1 筆波段交易
0x674bdf20a0f284d710bc40872100128e2d66bd3f -> 平均持有 90.0 天，共 1 筆波段交易
0x4f9dbb8d98866cd535ebceca207bc3062cb622a9 -> 平均持有 90.0 天，共 1 筆波段交易
0x0afe5074f71b31faca4632e63cfff80b5a982372 -> 平均持有 90.0 天，共 1 筆波段交易
0xba4e1ed8cd961b0a8926ed02079a1469c68f07fa -> 平均持有 89.0 天，共 1 筆波段交易
0xd053dcd7037af7204cece544ea9f227824d79801 -> 平均持有 89.0 天，共 1 筆波段交易
0x10bde0a716d1c8be11a6ec5bc95a7c00054fc097 -> 平均持有 89.0 天，共 1 筆波段交易
0x429cf888dae41d589d57f6dc685707bec755fe63 -> 平均持有 89.0 天，共 1 筆波段交易
0x4196d3479fda0ebed8b67ff8b3134dbf4c35f1a9 -> 平均持有 89.0 天，共 1 筆波段交易
0x9f8d8df26d5ab71b492ddce9799f432e36c289df -> 平均持有 89.0 天，共 1 筆波段交易
0x5d22045daceab03b158031ecb7d9d06fad24609b -> 平均持有 89.0 天，共 1 筆波段交易
0x25af1f2c3772d6f19aa

In [3]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time
import os
from dotenv import load_dotenv
load_dotenv()

# ---------------- 參數 ----------------
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
TARGET_WALLET = "0x86b792e6a20c8e8ef56ff4fc92aedcb62dbeefed"

# ---------------- Binance Kline ----------------
def get_binance_kline(symbol: str, interval: str, end_time: datetime, total_limit: int = 1000) -> pd.DataFrame:
    time.sleep(0.2)
    base_url = "https://api.binance.com/api/v3/klines"
    all_data = []
    end_timestamp = int(end_time.timestamp() * 1000)
    remaining = total_limit

    while remaining > 0:
        fetch_limit = min(1000, remaining)
        params = {
            "symbol": symbol.upper(),
            "interval": interval,
            "endTime": end_timestamp,
            "limit": fetch_limit
        }
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        if not data:
            break
        all_data = data + all_data
        end_timestamp = data[0][0] - 1
        remaining -= len(data)

    if not all_data:
        return pd.DataFrame(columns=["timestamp", "open", "high", "low", "close", "volume"])

    df = pd.DataFrame(all_data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
    df = df.drop_duplicates(subset="timestamp").sort_values("timestamp").reset_index(drop=True)
    return df[["timestamp", "open", "high", "low", "close", "volume"]]

# ---------------- Etherscan API ----------------
def get_wallet_transactions(wallet_address, start_time, end_time, erc20=False, token_list=None):
    wallet_address = wallet_address.lower()
    action = "tokentx" if erc20 else "txlist"

    url = f"https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": action,
        "address": wallet_address,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY
    }
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()
    if data['status'] != '1':
        return pd.DataFrame(columns=["timestamp","from","to","value","hash","token"])

    tx_list = []
    for tx in data['result']:
        tx_time = pd.to_datetime(int(tx['timeStamp']), unit='s')
        if tx_time < start_time or tx_time > end_time:
            continue
        if erc20:
            token = tx['contractAddress']
            if token_list and token.lower() not in [t.lower() for t in token_list]:
                continue
            value = int(tx['value']) / 10 ** int(tx.get('tokenDecimal', 18))
        else:
            token = None
            value = int(tx['value']) / 1e18
        tx_list.append({
            "timestamp": tx_time,
            "from": tx['from'],
            "to": tx['to'],
            "value": value,
            "token": token,
            "hash": tx['hash']
        })
    df_tx = pd.DataFrame(tx_list)
    if not df_tx.empty:
        df_tx['from'] = df_tx['from'].str.lower()
        df_tx['to'] = df_tx['to'].str.lower()
    return df_tx

# ---------------- 每筆交易都對應 signal ----------------
def map_wallet_to_signals(df_tx, wallet_address):
    if df_tx.empty:
        return pd.DataFrame(columns=["timestamp","direction","value","token","hash"])

    wallet_address = wallet_address.lower()
    df_signals = df_tx.copy()

    # direction: 1 = 收到, -1 = 轉出
    df_signals['direction'] = df_signals.apply(
        lambda row: 1 if row['to'] == wallet_address else -1, axis=1
    )

    return df_signals[["timestamp","direction","value","token","hash"]]

# ---------------- 統一抓取交易信號 ----------------
def get_all_signals(symbol: str, wallet_address=TARGET_WALLET, erc20=False, token_list=None, start_time=None, end_time=None):
    if end_time is None:
        end_time = datetime.utcnow()
    if start_time is None:
        start_time = end_time - timedelta(days=365)  # 預設抓最近一年

    df_tx = get_wallet_transactions(wallet_address, start_time, end_time, erc20=erc20, token_list=token_list)
    df_signals = map_wallet_to_signals(df_tx, wallet_address)
    return df_signals

def attach_kline_to_signals_any_interval(df_signals, symbol="WBTCUSDT", interval="1d", end_time=None):
    if df_signals.empty:
        return pd.DataFrame(columns=["timestamp","signal","value","open","high","low","close","volume"])

    if end_time is None:
        end_time = df_signals['timestamp'].max() + pd.Timedelta(minutes=1)

    # 計算需要抓多少根 K 線
    start_time = df_signals['timestamp'].min()
    interval_mapping = {
        "1m": 60,
        "3m": 180,
        "5m": 300,
        "15m": 900,
        "30m": 1800,
        "1h": 3600,
        "2h": 7200,
        "4h": 14400,
        "6h": 21600,
        "8h": 28800,
        "12h": 43200,
        "1d": 86400,
        "3d": 259200,
        "1w": 604800,
        "1M": 2592000
    }
    seconds_per_kline = interval_mapping.get(interval, 86400)
    total_seconds = (end_time - start_time).total_seconds()
    total_limit = int(total_seconds // seconds_per_kline) + 10  # 多抓幾根保險

    # 抓取 K 線
    df_kline = get_binance_kline(symbol, interval, end_time=end_time, total_limit=total_limit)

    if df_kline.empty:
        return pd.DataFrame(columns=["timestamp","signal","value","open","high","low","close","volume"])

    # 對應交易到最近不晚於交易時間的 K 線
    df_kline['timestamp'] = pd.to_datetime(df_kline['timestamp'])
    df_kline = df_kline.sort_values('timestamp')
    df_signals_sorted = df_signals.sort_values('timestamp')
    df_merged = pd.merge_asof(
        df_signals_sorted,
        df_kline,
        left_on='timestamp',
        right_on='timestamp',
        direction='backward'
    )

    # 將 direction 改名為 signal
    df_merged = df_merged.rename(columns={"direction": "signal"})
    df_merged = df_merged[["timestamp","signal","value","open","high","low","close","volume"]]

    return df_merged

# ---------------- 範例 ----------------
from datetime import datetime

end_time = datetime(2025, 8, 14)  # 往回抓到這個時間點
interval = "1d"
df_all_signals = get_all_signals("WBTCUSDT", wallet_address=TARGET_WALLET, erc20=True, token_list=["0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599"])
df_all_signals_kline = attach_kline_to_signals_any_interval(df_all_signals, symbol="WBTCUSDT", interval=interval, end_time=end_time)

display(df_all_signals_kline)
print(f"總交易筆數: {len(df_all_signals_kline)}")

,timestamp,signal,value,open,high,low,close,volume
0,2024-08-23 22:27:23,-1,0.001832,60323.13,64842.59,60278.38,63883.48,32.83512000
1,2024-08-25 00:07:35,-1,0.001476,64022.59,64875.12,63619.80,64036.55,54.00169000
2,2024-08-25 08:23:23,-1,0.000498,64022.59,64875.12,63619.80,64036.55,54.00169000
3,2024-08-28 01:00:11,1,0.001956,59318.39,60192.25,57774.31,58993.40,35.86500000
4,2024-08-28 09:46:11,1,0.001969,59318.39,60192.25,57774.31,58993.40,35.86500000
...,...,...,...,...,...,...,...,...
167,2025-08-03 15:06:47,1,0.000733,112500.34,114681.94,111878.26,114115.53,52.26306000
168,2025-08-10 13:45:11,-1,0.000575,116369.64,119174.36,116369.64,119148.13,53.88087000
169,2025-08-10 23:17:11,-1,0.000087,116369.64,119174.36,116369.64,119148.13,53.88087000
170,2025-08-11 02:28:59,-1,0.000860,119158.29,122124.70,118000.00,118562.51,87.61338000


總交易筆數: 172
